The Turán number of $C_7$
==============================

Forbidding all homomorphic images of C7: K4, F1, and F2

In [4]:
# Setting up three graph theory
K4 = ThreeGraphTheory(4, edges=[[0, 1, 2], [1, 2, 3], [0, 2, 3], [0, 1, 3]])
# C5 = ThreeGraphTheory.p(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0], [4, 0, 1]])
# ThreeGraphTheory.exclude([K4, C5])
F1 = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 4], [1, 3, 4]])
F2 = ThreeGraphTheory.pattern(6, edges=[[0, 1, 2], [0, 1, 5], [0, 2, 3], [0, 3, 4], [0, 4, 5], [1, 4, 5]])
ThreeGraphTheory.exclude([K4, F1, F2])
TG = ThreeGraphTheory

# Setting up colored theory
TwoColorTheory = combine("2Colors", Color0, Color1, symmetries=NoSymmetry)
TwoColorTheory.exclude([TwoColorTheory(1), TwoColorTheory(1, C0=[0], C1=[0])])
CTG = combine("2ColorThreeGraphs", TG, TwoColorTheory)

# Testing the generation of structures
print("Number of structures without K4, F1, F2")
print("and size 5: ", len(TG.generate(5)), " (should be 19)")
print("and size 6: ", len(TG.generate(6)), " (should be 371)")

print("\nNumber of 2 colored structures without K4, F1, F2")
print("and size 5: ", len(CTG.generate(5)), " (should be 302)")
print("and size 6: ", len(CTG.generate(6)), " (should be 16807)")
TG.printlevel(0)

Number of structures without K4, F1, F2
and size 5:  19  (should be 19)
and size 6:  371  (should be 371)

Number of 2 colored structures without K4, F1, F2
and size 5:  302  (should be 302)
and size 6:  16807  (should be 16807)


In [5]:
a31 = TG.optimize(TG(3, edges=[[0, 1, 2]]), 6, exact=True, denom=2**20, file="certificates/prop_3_1_c7")
print("The initial upper bound on the Turan density from Proposition 3.1 is {} ~= {}".format(a31, a31.n()))

The initial upper bound on the Turan density from Proposition 3.1 is 60891/131072 ~= 0.464561462402344


In [6]:
# Generate good partition
target_size = 6
all_graphs = TG.generate(target_size, TG(3, ftype=[0, 1, 2], edges=[[0, 1, 2]]))

edge = TG(3, edges=[[0, 1, 2]])
target_typed = 0
for ii, flag in enumerate(all_graphs):
    nfp = flag.not_ftype_points()
    if flag.subflag(points=nfp, ftype_points=[]) != edge:
        continue
    A = 0
    for xx in nfp:
        touch = len(flag.subflag(points=[xx]).blocks("edges")) - 1
        if touch not in [0, 1]:
            A += 1
    if A == 2:
        target_typed += flag
target_flag = target_typed.project()

# Average degree lower bound 
positives = [TG(3, edges=[[0, 1, 2]]) - 4641/10000]

# Optimize
ratio = TG.optimize(target_flag, target_size, maximize=False, 
                    positives=positives, exact=True, denom=2**20, 
                    file="certificates/prop_3_2_c7"
                   )
a32 = ratio / a31
print("The max-cut ratio returned by Proposition 3.2 is at least {} ~= {}".format(a32, a32.n()))

The max-cut ratio returned by Proposition 3.2 is at least 2013187661/4871280000 ~= 0.413276933578033


In [7]:
target_size = 6

# Edge with correct colors
C = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2])
# Missing edge with good colors
M = CTG(3, edges=[], C0=[0, 1], C1=[2])
# Pointed edge with correct colors, point from color 0
Cp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[0])
# Pointed edge with correct colors, point from color 1
Cp1 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[2])


# Edge with bad colors all in color 0
B000 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[])
# Edge with bad colors, looking the wrong way
B011 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2])
# Pointed edge with bad colors, point from moving 1 -> 0
Bp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[], ftype=[0])
# Pointed edge with bad colors, point from moving 0 -> 1
Bp1 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2], ftype=[2])

# Degree equality conditions for various type pairs
degeq_00 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1])
degeq_01 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0], C1=[1]) - \
           CTG.p(4, ftype=[0, 1], edges=[[1, 2, 3]], edges_m=[[0, 2, 3]], C0=[0], C1=[1])
degeq_11 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1])

# Assumptions used in the proof
assums = [
    Cp0 - Bp0, Cp1 - Bp1, # Local optimality
    degeq_00, -degeq_00, degeq_01, -degeq_01, degeq_11, -degeq_11, # Degree regularity
    C - 2/5 # Best fit assumption
]

# Close rational approximation for gamma: 2521/5432
gamma = continued_fraction(2*sqrt(3) - 3).convergent(7)
# Optimal construction and its derivatives
symbolic_constr = CTG.blowup_construction(target_size, ["X0", "X1"], edges=[[0, 0, 1]], C0=[[0]], C1=[[1]]).set_sum()
ders = symbolic_constr.derivatives([gamma])

/home/levente/sage/src/sage/algebras/combinatorial_theory.py:2919: UserWarning: The pattern is initialized with optional relations inside 
            the ftype. Those relations will be missing in the resulting pattern.
  return Pattern(self, n, ftype_points, **blocks)


In [8]:
# First solving the sdp
CTG.solve_sdp(B000 + B011 - (1-1/4000)*M, target_size, maximize=True,
              positives=assums, construction=ders, 
              file="certificates/prop_3_3_c7_sdp")

Base flags generated, their number is 16807
The relevant ftypes are constructed, their number is 33
Block sizes before symmetric/asymmetric change is applied: [37, 37, 37, 128, 128, 128, 128, 128, 94, 94, 94, 94, 94, 94, 94, 94, 78, 78, 78, 78, 78, 78, 78, 78, 78, 36, 36, 36, 36, 36, 36, 36, 36]


Done with mult table for Ftype on 4 points with edges=(013 023 123), C0=(0 1 2 3), C1=(): : 33it [00:09,  3.32it/s]


Adjusting table with kernels from construction
Tables finished


Done with positivity constraint 8: 100%|██████████| 9/9 [02:41<00:00, 17.90s/it]


Constraints finished
Running SDP. Used block sizes are [26, 10, 36, 26, 10, 21, 106, 39, 88, 56, 72, 40, 88, 21, 106, 28, 66, 58, 36, 28, 66, 58, 36, 58, 36, 28, 66, 58, 36, 28, 66, 34, 44, 48, 30, 50, 28, 78, 34, 44, 33, 44, 50, 28, 48, 30, 34, 44, 16, 20, 16, 20, 26, 10, 26, 10, 26, 10, 26, 10, 15, 20, 16, 20, -16807, -304]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 4.07e-02 Pobj: -7.2634649e+00 Ad: 2.82e-02 Dobj:  1.0037747e+02 
Iter:  2 Ap: 4.94e-02 Pobj: -1.7394525e+01 Ad: 7.52e-02 Dobj:  8.1652494e+01 
Iter:  3 Ap: 1.15e-01 Pobj: -4.1776578e+01 Ad: 1.29e-01 Dobj:  8.8920211e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.9796860e+02 Ad: 3.00e-01 Dobj:  5.9015251e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -2.2498211e+02 Ad: 8.28e-01 Dobj:  9.5789416e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -2.4297614e+02 Ad: 9.26e-01 Dobj:  6.8913116e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -2.6655755e+02 Ad: 9.10e-01 Dobj:  8.5230265e-02 
Iter:  8 Ap: 1.00e+00 Pobj: -3.217013

1.1184092540972301e-10

In [9]:
# Second, rounding the sdp solution
CTG.round_solution("certificates/prop_3_3_c7_sdp", certificate_file="certificates/prop_3_3_c7", 
                   denom=2**10 * 225, slack_threshold=1e-6)

Starting the rounding of the result
Flattening X matrices
This took 59.48841214179993s
Correcting flat X matrices
Dimensions:  (399, 57021)
This took 2.426697254180908s
Unflattening X matrices
This took 0.0007405281066894531s
Calculating resulting bound


100%|████████████████████████████████████████| 33/33 [5:36:57<00:00, 612.65s/it]


This took 20219.04615688324s
Final rounded bound is 0


0